# Malware Analysis & Triage Kit
This notebook performs the initial stages of immediate malware triage.

## How To
This notebook will walk you through the stages of initial analysis.

It automatically downloads malware samples from MalwareBazaar or you can choose to manually add them to the Dropbox directory.

At the end of this process, you will have a triage report in the `saved-specimens` diretory. This report includes findings from initial triage, including the defanged specimen in a password-proteced Zip file and static analysis artifacts.

If you don't want to enter your API credentials every time, create a `.blue-jupyter` file in your home directory and add them there with the following format:

```
MB_API_KEY=ABCDEFG
API_KEY_2=HIJKLMN
```

# Imports and Setup

In [1]:
# Imports
from hashlib import *
import sys
import os
import subprocess
from getpass import getpass
from virus_total_apis import PublicApi as VirusTotalPublicApi
import json
from MalwareSample import *
from MalwareBazaar import *
from Utils import *
from pprint import pprint
import os.path
from time import sleep
from datetime import datetime
import requests

## Malware Bazaar

### Malware Bazaar Setup

In [2]:
API_URL = "https://mb-api.abuse.ch/api/v1/"

MB_API_KEY = Utils.get_credential("MB_API_KEY")
if not MB_API_KEY:
    MB_API_KEY = getpass("Enter Malware Bazaar API Key (blank if none): ")

### Malware Download

In [3]:
if not MB_API_KEY:
    print(info + "No MB_API_KEY. Skipping.")
else:
    to_download_count = input("Number to download (Max of 100): " )

    if to_download_count:
        to_download_count = int(to_download_count)
    else:
        to_download_count = 100;

    print(info + "Starting download...")
    response_data = MalwareBazaar.get_recent_list(API_URL, MB_API_KEY)
    
    current_download_count = 0
    for i in response_data:

        directory = os.getcwd() + "/dropbox/"
        sha256_hash = i["sha256_hash"]
        
        if i["file_type"] != "exe":
            print(info + "Skipping. Not exe file: " + sha256_hash)
            continue

        try:
            if subprocess.check_output('grep -i "' + sha256_hash + '" processed.txt', stderr=subprocess.DEVNULL, shell=True):
                print(info + "Skipping. Already processed: " + sha256_hash)
                continue
        except:
            if os.path.exists(directory + sha256_hash + ".exe"):
                print(info + "Skipping. Already downloaded: " + sha256_hash)
                continue
                
        MalwareBazaar.download_malware(sha256_hash, API_URL, MB_API_KEY)

        filename = sha256_hash + ".zip"
        Utils.unzip_file(directory, filename, "infected")
        
        print(info + "Downloaded: " + sha256_hash)    
        current_download_count += 1
        if current_download_count >= to_download_count:
            break

    print(info + "Finished downloading " + str(current_download_count) + " files.")

Number to download (Default 100):  1


[*] Starting download...
[*] Fetching list...
[*] Downloaded: 76257feb8d753a419e11bd0672eac3d236ac990d8ba8baf7ec44c5f1f3eac591
[*] Finished downloading 1 files.


### Check Dropbox and Saved-Specimens

In [ ]:
MalwareSample.check_dir("dropbox")
MalwareSample.check_dir("saved-specimens")
empty = MalwareSample.is_dir_empty("dropbox")
if empty:
    print(r"  \\--> " + recc + "Put some samples in the dropbox!")

### Enumerate Samples in the Dropbox

In [ ]:
samples=!ls dropbox/*
for s in samples:
    print(info + "Sample: " + s)

In [ ]:
sample_obj = [MalwareSample(s) for s in samples]

### Create a Saved Specimen directory for the specimen(s)

In [ ]:
for obj in sample_obj:
    saved_sample_name = MalwareSample.create_specimen_dirs(obj.sample_name)
    obj.saved_sample_name = saved_sample_name

### Defang Sample

In [ ]:
for obj in sample_obj:
    sample_path = MalwareSample.move_and_defang(obj.sample_name, obj.saved_sample_name)
    obj.sample_path = sample_path

---

## File Hashes

### SHA256 Sum

In [ ]:
for obj in sample_obj:
    hash = MalwareSample.get_sha256sum(obj.sample_path, obj.saved_sample_name)
    obj.sha256sum = hash
    print(info + obj.sample_name + ": " + obj.sha256sum)

---

## String Analysis

### StringSifter
StringSifter is a FLARE developed tool that uses an ML model to rank a binary's strings by relevance to malware analysis.

In [ ]:
length = int(input(recc + "Input your desired minimum string length [default is 4, 6-8 is recommended] > "))

In [ ]:
for obj in sample_obj:
    MalwareSample.pull_strings(length, obj.saved_sample_name, obj.sample_path)
    
print(info + "Done.")

## VT Analysis
Submit samples to Virus Total and generate a malicious confidence level.

In [ ]:
VT_API_KEY = Utils.get_credential("VT_API_KEY")
if not VT_API_KEY:
    VT_API_KEY = getpass("Enter VirusTotal API Key (blank if none): ")

In [ ]:
if VT_API_KEY:
    vt = VirusTotalPublicApi(VT_API_KEY)
else:
    print(info + "No VT API Key. Skipping...")

Note: If there are more than 4 samples in the dropbox, hashes are submitted with a sleep of 16 seconds to remain under the public API rate limit. So hit go, grab a beverage of choice, stretch out and relax. This could be a while depending on how many samples you're submitting.

In [ ]:
if VT_API_KEY:
    for obj in sample_obj:
        print(info + obj.sample_name + ":")
        print(r"  \\--> " + info + "SHA256sum: " + obj.sha256sum)
        res = vt.get_file_report(obj.sha256sum)
        conf = malicious_confidence(res)
        print(r"  \\--> " + info + "Confidence level: " + str(conf))
        crit_level =  determine_criticality(conf)
        obj.criticality = crit_level
        
        if len(sample_obj) >= 5:
            sleep(16)
        
else:
    print(info + "No VT API Key. Skipping...")
    
print(info + "Done")


## Zip and Password Protect

In [ ]:
for obj in sample_obj:
    zip_file = MalwareSample.zip_and_password_protect(obj.sample_path, obj.saved_sample_name)
    MalwareSample.delete_unzipped_sample(obj.sample_path, zip_file)

## Delete Downloads

In [ ]:
downloads=!ls dropbox/*
if "No such file or directory" in downloads[0]:
    print(info + "Dropbox directory is empty.")
else:
    with open("processed.txt", "a") as f:
        for d in downloads:
            print(info + "Removed: " + d)
            os.remove(d)
            sha256_sum = os.path.splitext(os.path.basename(d))[0]
            f.write(str(datetime.now()) + " " + sha256_sum + "\n")

---

### Debug Object Vars

In [ ]:
for obj in sample_obj:
    pprint(vars(obj))